In [10]:
import os
from datetime import date

import requests
import pandas as pd
import altair as alt

In [11]:
API_KEY = os.environ.get('BLS_API_KEY')

In [12]:
BASE_URL = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'

In [13]:
# This series ID denotes all industries, all U.S. regions, seasonally adjusted quit rate
# REF: https://www.bls.gov/help/hlpforma.htm#jt
JOLTS_SERIES = 'JTS000000000000000QUR'

In [14]:
# the API only allows you to grab 10 years of data at a time,
# so breaking this into chunks for future resiliency
first_year = 2001
this_year = date.today().year

start_years = list(range(first_year, this_year+1, 10))
year_ranges = [(x, x+9) if x+9 < this_year else (x, x) for x in start_years]
print(year_ranges)

[(2001, 2010), (2011, 2020), (2021, 2021)]


In [15]:
jolts_data = []

In [16]:
for years in year_ranges:
    data = {
        'registrationkey': API_KEY,
        'seriesid': JOLTS_SERIES,
        'startyear': years[0],
        'endyear': years[1]
    }
    r = requests.post(BASE_URL, data=data)
    r.raise_for_status()
    this_data = r.json()['Results']['series'][0]['data']
    jolts_data += this_data

In [17]:
jolts_data.sort(key=lambda x: (x['year'], x['period']))

In [18]:
df = pd.DataFrame(data=jolts_data)[['year', 'period', 'value']]

In [19]:
df.head()

,year,period,value
0,2001,M01,2.4
1,2001,M02,2.3
2,2001,M03,2.3
3,2001,M04,2.4
4,2001,M05,2.3


In [20]:
# a few integrity checks
df.year.value_counts().sort_index()

2001    12
2002    12
2003    12
2004    12
2005    12
2006    12
2007    12
2008    12
2009    12
2010    12
2011    12
2012    12
2013    12
2014    12
2015    12
2016    12
2017    12
2018    12
2019    12
2020    12
2021     8
Name: year, dtype: int64

In [21]:
# a few integrity checks
df.period.value_counts().sort_index()

M01    21
M02    21
M03    21
M04    21
M05    21
M06    21
M07    21
M08    21
M09    20
M10    20
M11    20
M12    20
Name: period, dtype: int64

In [22]:
# a few integrity checks
print(df.value.min())
print(df.value.max())

1.2
2.9


In [24]:
# create a date column for the chart
df['date'] = df.apply(lambda row: row['year'] + '-' + row['period'].lstrip('M') + '-01', axis=1)
df['date'] = pd.to_datetime(df['date'])

In [25]:
df.head()

,year,period,value,date
0,2001,M01,2.4,2001-01-01
1,2001,M02,2.3,2001-02-01
2,2001,M03,2.3,2001-03-01
3,2001,M04,2.4,2001-04-01
4,2001,M05,2.3,2001-05-01


In [26]:
# adjust the numbers for percent display -- divide by 100
# to reverse the multiplication by 100 that happened upstream
df['value'] = df['value'].astype(float) / 100.0

In [27]:
alt.Chart(df).mark_area(
    color='lightblue',
    interpolate='step-before',
    line=True
).encode(
    x=alt.X('date:T', axis=alt.Axis(title='')),
    y=alt.Y('value:Q', axis=alt.Axis(title='Quit rate', format='%')),
).properties(
    width=800
)

alt.Chart(...)